In [1]:
import sys
sys.path.append('../')   # If you move this notebook to another location, change this path to point to the root directory of the solutions project
import importlib

import pandas as pd
from pathlib import Path
from integrations import waste_integration as wi
from model import sma

wi.load_default_values()

pd.set_option('display.float_format', lambda x: f"{x:03.5f}".rstrip('0'))

In [11]:
importlib.reload(wi)
wi.load_default_values()

Table 1: Composite Mt MSW from multiples sources: Using data from the Intergovernmental Panel on Climate Change (IPCC), 2006; Hoornweg & Bhada-Tata (2012);Bahor, et al(2009); Hoornweg et al (2015); and the International Energy Agency (IEA),2016, using a best fit trend analysis and extrapolation, a composite measure of prognosticated MSW from 2015-2060

Table 2: Composite Mt MSW Organic Fraction: Using waste fraction data from the Intergovernmental Panel on Climate Change (IPCC), 2006 and Hoornweg & Bhada-Tata (2012) a forecast for the organic fraction of MSW is devloped for Drawdown regions and the World.  Food waste and "green material" is considered organic fraction.  Soiled paper may be considered "green material" in some regions, but paper waste is allocated elsewhere.  Green material includes landscape waste and certain debris.  This forecast becomes the TAM for Composting and is directly affected by reductions in food waste and an increase in Bioplastics production and a simulataneous, but distinct forecast in compostability of bioplastic.  When bioplastic is considered compostable it moves to the organic fraction.

In [12]:
wi.get_compost_organic_fraction().head()

,World,OECD90,Eastern Europe,Asia (Sans Japan),Middle East and Africa,Latin America,China,India,EU,USA
Year,,,,,,,,,,
2014,804.141975,315.515461,22.683366,129.094524,232.848243,103.766553,105.421726,130.151850,96.544192,63.433059
2015,852.016766,334.299752,24.033825,136.780199,246.710920,109.944320,111.698035,137.900473,102.291974,67.209562
2016,888.931264,348.783631,25.075115,142.706341,257.399924,114.707770,116.537466,143.875152,106.723879,70.121485
2017,916.580932,359.632334,25.855062,147.145135,265.406192,118.275686,120.162294,148.350301,110.043460,72.302571
2018,947.556992,371.786190,26.728839,152.117938,274.375654,122.272840,124.223206,153.363833,113.762404,74.746053


Table 3: Composite Mt MSW Recycleable Fraction (This is TAM for Recycling); Does NOT Include Paper. This is developed from multiple sources including  the Intergovernmental Panel on Climate Change (IPCC), 2006; Hoornweg & Bhada-Tata (2012);Bahor, et al(2009); Hoornweg et al (2015).

In [13]:
wi.get_waste_recyclable_fraction().head()

,World,OECD90,Eastern Europe,Asia (Sans Japan),Middle East and Africa,Latin America
Year,,,,,,
2014,627.405333,185.217583,29.540125,90.921071,248.202177,73.523749
2015,657.888723,194.216646,30.975375,95.338602,260.261437,77.096006
2016,676.360744,199.669808,31.845093,98.015494,267.568987,79.260687
2017,693.981010,204.871522,32.674708,100.568952,274.539582,81.325553
2018,710.783477,209.831812,33.465818,103.003898,281.186654,83.294583


Table 4: Composite Mt MSW Remainder Fraction (This should include Paper + others not included in Recyclable).  This is calculated by subtracting Table 2 and Table 3 values from Table 1 values. I.e., Total MSW - Organic fraction - Recycleable fraction = Remainder fraction.

In [14]:
wi.get_waste_remainder().head()

,World,OECD90,Eastern Europe,Asia (Sans Japan),Middle East and Africa,Latin America,China,India,EU,USA
Year,,,,,,,,,,
2014,341.786025,30.608280,124.709843,527.369564,-225.738320,46.273854,NaN,NaN,NaN,NaN
2015,359.627682,20.827159,115.220216,545.707916,-237.845310,45.520972,NaN,NaN,NaN,NaN
2016,367.167261,9.023387,112.115908,559.526143,-243.761276,45.302372,NaN,NaN,NaN,NaN
2017,383.063482,0.683960,109.450226,574.426362,-246.735360,46.233763,NaN,NaN,NaN,NaN
2018,394.760274,-9.126417,106.860629,588.394889,-250.415804,46.690431,NaN,NaN,NaN,NaN


Table 5: Plastic Fraction:  Not all plastic produced each year enters MSW.  This estimates plastic in MSW based on benchmark from World Bank 2018. 12% of MSW is plastics.

Table 6: Global Waste fractions Pre-Integration:  This is calculated by dividing each table of values corresponding to each MSW fraction by the table of values in Table 1.  It is presented here for benchmarking and callibration

## INTEGRATION -  1st Order and 2nd Order

Table 7: Reduction in Food Waste: This is the adoption of reduction of food waste measure in Mt (includes only food waste reduction in 'Distribution' and 'Consumption' stages of Food System

Table 8: Reduce Total MSW by Reduction in Food Waste: First step in Waste and Materials integration is to subtract reduction in food waste from total MSW forecast. Table 1 - Table 7 values.

Table 9: Reduce Organic MSW by Reduction in Food Waste: Substracting the reduction in food waste from the organic fraction revises the TAM for Composting by limiting the amount of organic material in MSW that can be composted. Table2 - Table 7 values.

Table 10. World Adoption of Bioplastics: This is the global adoption of bioplastics.  Each year bioplastics are produced it is assumed that an equivalent amount will enter MSW.

Table 11: % of Bioplastics assumed to be compostable:  

Table 12: Total World MSW after reduction in Food Waste:  Restatement of Table 8.

Table 13: Total World Organic MSW after reduction in Food Waste and with addition of a % of Bioplastics.  Add compostable Bioplastics disposed into MSW into Organic Fraction.  This impacts the TAM for Composting.  The calculation is Table 9 + Table 10 multiplied by values in Table 11

Table 14: Total world recyblable MSW after subraction of % bioplasitcs (assumes material would otherwise have been recylable plastic)

Table 15: Total World Remainder MSW after subtraction of reduced food waste and fraction mix change due to bioplastics (not Organics and Not Recyclables) DOES include Paper.  Table 12- Table 13 - Table 14

Table 16: Total World Plastic MSW (after subtraction of Bioplastics).  Table 5 - Table 10. Assumption - equivalent amount of bioplastics would make it to MSW (**this might be revised in the future to assume same MSW rate as plastics).

Table 17: Global Waste fractions Post-1st and 2nd Order Integration.  Characterizing world MSW fractions after subtraction of reduced food waste and mix shift from Bioplastics.  Used for benchmarking and calibration.

## Integrated PDS 1 Adoptions in Consideration of Integrated MSW TAM (post 2nd Order Integration)

Table 18: PDS 1 Adoption of Compost.  If % of Organic MSW fraction exceeds 100% than feedstock limitation applied and adoption is curtailed to match available feedstock.  % is calculated by Table 18 / Table 13.

Table 19: PDS 1 Adoption of Commercial and Household Recycling.   If % of Recycleable MSW fraction exceeds 100% than feedstock limitation applied and adoption is curtailed to match available feedstock.  % is calculated by Table 19 / Table 14.

Table 20: Global MSW Fractions after Composting, Household and Commercial Recycling, but before Paper Recycling.  Used for Benchmarking. 

Table 21: PDS 1 Recycled Paper Adoption.  Benchmark of Remainder Portion of Global MSW presented to benchmark that Adoption of Recycled Paper does not exceed paper available in anuual MSW.

Table 22: Remaining MSW (Mt) after PDS 1 Composting, Household and Commercial Recycling and Recycled Paper adoption have reduced amount of global MSW.  Fractions are used for benchmarking as percentages and for adjusting LHV for WTE.

Table 23: MSW to WtE - Pre-Integration.  These are inputs to sheet WTE Tonnes and Composition.  The available world waste feedstock for Waste to Energy conversaion is determined as the minimum of the results from calculations on sheet WTE Tonnes and Composition based on TWh adoption scenarios for WTE or the remaining waste available per table 22.  The average LHV is calculated per the changing mix of waste materials calculated in table 22

Table 24: Remaining MSW (Mt) to be Landfilled.  The amount of material landfilled is calculated be subtracting the values in Table 23 from the values in Table 22.  

## INTEGRATION -  MSW Fraction Analysis with Attention to Paper fraction, Organic fraction/Degradable Carbon Fraction

Table 25: Take total food waste as a % of organic fraction.  Add in Bioplastics.  Assess the remainder.  This table is used for benchmarking.

Table 26 Analysis of 'Remainder' Fraction post 2nd Order Integration to evaluate possible 'TAM' for Recycle Paper.  

Table 27: Solutions Impact on MSW

Table 28: MSW Fraction Detail Post Materials Solutions

Table 29: Solutions Impact on MSW

# Extraction Logic

Notes:

List of sheets:

* Table of Contents, Overview
* MSW Integration Calcs PDS1, PDS2, PDS3
* WTE Tonnes and Composition
* WTE and LM Analysis
* Landfill Methane
* Food System Summary
* Paper
* Compost
* H&C Recycling
* Bioplastics
* Refrigerant Management
* Total MSW TAM Data_Updated2020
* Total MSW Summaries_Updated2020
* Background sheets:  What a Waste 2.0, IPCC (2019 Refinement), UNEP Global Waste Management, USA Data- EPA, India Data, EU Data
* Total MSW Sumamaries (Archive)
* Total MSW TAM Data (Archive)

Analysis of formula references:
```
    (PDS1, PDS2, PDS3) <--> (WTE Tonnes & Composition, Paper, Compost, Recycling)
    (PDS1, PDS2, PDS3) <-- Food System Summary
    PDS1 <--> Bioplastics
    (WTE Tonnes & Composition <--> WTE and LM Analysis <--> Landfill Methane)  <-- (PDS1, PDS2, PDS3)
    Refrigerant Management  <-->  None
```
That is, PDS1,2,3 are both input and output for the topic tabs, and the Waste & Landfill Methand tabs are a tightly bound cluster.  Refrigerant Management is isolated, as are all the background sheets, and the MSW TAM Update.

For some reason Bioplastics are tied only with PDS1?

The instruction document is quite detailed and helpful.  We'll mostly follow along.

In [2]:
import openpyxl
from model import sma
from tools import sma_xls_extract
import warnings

warnings.simplefilter("ignore")  # silence openpyxl warnings

In [7]:
import importlib
importlib.reload(sma)

<module 'model.sma' from 'C:\\Working\\solutions\\integrations\\..\\model\\sma.py'>

In [8]:
excel_file = Path(r'C:\Working\Integrations\Materials and Waste Integration Analysis_June2020.xlsm')
wb = openpyxl.load_workbook(excel_file, data_only=True, keep_links=False)

### Waste TAM

In [9]:
# Step one:  Save the "Waste TAM" to a data file.
# (Note: the instructions say to update the Waste TAM from another integration step; for now 
# are just going to assume this has been done already)
wastetam = sma_xls_extract.extract_sma(wb, 'Total MSW TAM Data_Updated2020', year_col='B', 
                                       start_row=41, use_functional_unit=True)

In [10]:
wastetam.region_cases

{'World': {'Base Cases': ['S1', 'S2'], 'Conservative Cases': ['S3']},
 'OECD90': {'Base Cases': ['S1', 'S2'], 'Ambitious Cases': ['S4']},
 'Eastern Europe': {'Base Cases': ['S1', 'S2'], 'Ambitious Cases': ['S4']},
 'Asia (Sans Japan)': {'Base Cases': ['S1', 'S2'], 'Ambitious Cases': ['S4']},
 'Middle East and Africa': {'Base Cases': ['S1', 'S2', 'S5'],
  'Ambitious Cases': ['S4']},
 'Latin America': {'Base Cases': ['S1', 'S2'], 'Ambitious Cases': ['S4']},
 'China': {'Base Cases': ['S6'], 'Ambitious Cases': ['S7']},
 'India': {'Base Cases': ['S6', 'S8'], 'Ambitious Cases': ['S7', 'S9']},
 'EU': {'Base Cases': ['S6', 'S10'], 'Ambitious Cases': ['S7']},
 'USA': {'Base Cases': ['S6', 'S11'], 'Ambitious Cases': ['S7']}}

In [11]:
wastetam.write(r'data\msw','waste_tam')

In [7]:
wb.close()

In [3]:
wt2 = sma.SMA.read(r'data\msw',"waste_tam")

In [6]:
wt2.sources['S1'].data.head()

,Year,World,OECD90,Eastern Europe,Asia (Sans Japan),Middle East and Africa,Latin America
0,2012,1536.666667,520.994064,174.333333,714.887215,227.431963,215.467580
1,2013,1655.000000,526.167694,175.633333,731.136187,241.372032,219.515868
2,2014,1773.333333,531.341324,176.933333,747.385160,255.312100,223.564155
3,2015,1891.666667,536.514954,178.233333,763.634132,269.252169,227.612443
4,2016,2010.000000,535.700000,178.200000,769.100000,302.400000,230.800000


### Step Two: Subject Sources as SMAs

In [3]:
# This is really just manual work, copying the current values of the tables, e.g. Compost!I3:J52 into SMA data format.
# I suppose I could write code to do this, but it doesn't seem worth it.
# Key note:  I change the number format to six digits precision before copying.
# Other Note:  The instructions to the model say to get these data from the corresponding model (e.g. composting), 
# but this is not a part of those models we are saving or representing currently.
# Example result:
cf = sma.SMA.read( Path("data/msw"), "organic_compost_sma" )
cf.summary(case='S1').head()

,World
Year,
2012,44.831749
2013,45.099481
2014,45.346352
2015,45.573771
2016,46.000000
